<a href="https://colab.research.google.com/github/hwangjunseong/my-first-website/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9Dteam_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사용할 라이브러리

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import numpy.linalg as lin

from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier


데이터 모으기 , 종속변수 벡터와 독립변수 벡터를 분리하기

In [31]:
train = pd.read_csv("/content/fashion-mnist_train.csv")
test = pd.read_csv("/content/fashion-mnist_test.csv")



X_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values
X_test= test.iloc[:,1:].values
y_test=test.iloc[:,0].values

y_train = np.array(y_train).ravel()#2차원 배열을 1차원 배열로 변경
y_test = np.array(y_test).ravel()
#ytrain.reshap(행의개수, 열의개수)
print(y_train)

[2 9 6 ... 8 8 7]


pca를 진행하기전에 표준화 보통 [-3,3]에 값이 위치, 또는 정규화[0, 1] 둘 중 하나 선택

In [ ]:

#scaler_minmax=MinMaxScaler()
#X_train_std = scaler_minmax.fit_transform(X_train)
#X_test_std = scaler_minmax.transform(X_test)

In [25]:
#입렦값으로 2차원 배열을 요구한다

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
#print(np.mean(X_train_std))

PCA

In [26]:

# PCA 모델 생성
pca = PCA(n_components=100)
X_train_reduced = pca.fit_transform(X_train_std)
X_test_reduced = pca.transform(X_test_std)
#print(X_train_reduced)

모델 선택,그리디 서치로 최적의 parameter찾기,  앙상블


In [28]:
param_grid = {
    'max_depth': [10, 20, 30],  # 트리의 최대 깊이
    'criterion': ['gini', 'entropy']  # 분할 품질을 측정하는 기준
}
grid_search=GridSearchCV(RandomForestClassifier(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train_reduced,y_train) #여기서 y_train이 1차원 배열이여야함
rf = grid_search.best_params_
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

KeyboardInterrupt: ignored

In [2]:

#넣을 후보들
param_grid={"criterion":["gini", "entropy", "log_loss"], "max_depth":[2, 4, 8]}
# 모델
grid_search=GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train_reduced, y_train)
# 데이터 평가
dt = grid_search.best_params_
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

NameError: ignored

In [8]:
param_grid = {
    'C': [0.1, 1, 10],  # 규제 매개변수 (작을수록 규제가 강함)
    'kernel': ['linear', 'rbf', 'poly'],  # 커널 종류

}
grid_search=GridSearchCV(SVC(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train_reduced, y_train)
svr = grid_search.best_params_
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

KeyboardInterrupt: ignored

In [ ]:

param_grid = {
    'n_neighbors': [3, 5, 7],  # 이웃의 수
    'weights': ['uniform', 'distance'],  # 가중치 옵션
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # 이웃 검색에 사용되는 알고리즘
    'p': [1, 2],  # 거리 측정에 사용되는 매개변수 (1: 맨해튼 거리, 2: 유클리드 거리)
    'metric': ['minkowski', 'manhattan', 'euclidean']  # 거리 측정 메트릭
}

# 모델
grid_search=GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train_reduced, y_train)
knn = grid_search.best_params_
# 데이터 평가
print("Best Parameter: {}".format(grid_search.best_params_))
print("Best Cross-validity Score: {:.3f}".format(grid_search.best_score_))

In [ ]:
neigh_model = KNeighborsClassifier(n_neighbors=knn['n_neighbors'])
rnf_model = RandomForestClassifier(criterion=rf['criterion'] ,random_state=42)
svm_model = SVC(kernel = svr['kernel'])


model = VotingClassifier(
    estimators=[('nm', neigh_model), ('rf', rnf_model), ('svc', svm_model)], voting='hard')
model.fit(X_train_reduced, y_train)

KeyboardInterrupt: ignored

test데이터로 예측

In [ ]:

y_pred = model.predict(X_test_reduced)
print(X_test_reduced)
print(y_pred)

혼동행렬 사용하여 정확도 예측

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

예측된 클래스 값을 testResult.txt 파일에 저장

In [ ]:

with open("testResult.txt", "w") as f:
    for index, prediction in enumerate(y_pred):
      index_str = f"{index:05d}" #5자리의 길이로 표시하고, 빈 자리는 0으로 채우는 문자열을 생성
      result = f"{index_str} {int(prediction)}"
      f.write(result + "\n")


In [ ]:
with open("testResult.txt", "r") as f:
    content = f.read()

print(content)